# Plot Maps of the Service Territory for Each Balancing Authority


In [ ]:
# Start by importing the packages we need:
import os

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable


## Set the Directory Structure


In [ ]:
# Identify the data input and image output directories:
data_input_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2023_applied_energy/data'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2023_applied_energy/figures/Balancing_Authorities/'


## Create the Function to Plot the Service Territory for a Given BA


In [ ]:
def plot_ba_service_territory(ba_to_plot: str, data_input_dir: str, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Read in the county shapefile and reassign the 'FIPS' variable as integers:
    counties_df = gpd.read_file(os.path.join(data_input_dir, r'county_shapefiles', r'tl_2020_us_county.shp')).rename(columns={'GEOID': 'County_FIPS'})
    counties_df['County_FIPS'] = counties_df['County_FIPS'].astype(int)

    # Read in the BA mapping file:
    ba_mapping_df = pd.read_csv((os.path.join(data_input_dir, 'BA_Service_Territory.csv')), index_col=None, header=0)

    # Merge the ba_mapping_df and counties_df together using county FIPS codes to join them:
    counties_df = counties_df.merge(ba_mapping_df, on='County_FIPS', how='left')

    # Subset to only the BA you want to plot:
    counties_subset_df = counties_df.loc[counties_df['BA_Code'] == ba_to_plot]
    
    # Read in the state shapefile:
    states_df = gpd.read_file(os.path.join(data_input_dir, r'state_shapefiles', 'tl_2022_us_state.shp')).rename(columns={'NAME': 'State'})
    
    # Create the figure:
    fig, ax = plt.subplots(1, 1, figsize=(25, 10))
    plt.rcParams['font.size'] = 18
    ax1 = counties_subset_df.plot(ax=ax, edgecolor='black', linewidth=1)
    ax2 = states_df.boundary.plot(ax=ax, edgecolor='black', linewidth=1.5)
    ax.set_xlim(-126.5, -66);
    ax.set_xlabel('Longitude', fontsize=18); 
    ax.set_ylim(24, 50); 
    ax.set_ylabel('Latitude', fontsize=18);
    ax.set_title((ba_to_plot + ' Service Territory'), fontsize=21)

    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       plt.savefig(os.path.join(image_output_dir + 'BA_Service_Territory_' + ba_to_plot + '.png'), dpi=image_resolution, bbox_inches='tight')
       plt.close()


## Loop over the BAs and Make the Plot


In [ ]:
# Read in the BA-to-interconnection mapping file:
ba_mapping_df = pd.read_csv((os.path.join(data_input_dir, 'BA_to_Interconnection_Mapping.csv')), index_col=None, header=0)
    
# Make a list of all of the BAs in the 'output_df' created above:
bas = ba_mapping_df['BA_Code'].unique()
         
# Loop over the BAs and make the plot for each BA:
for i in range(len(bas)):
    plot_ba_service_territory(ba_to_plot = bas[i],
                              data_input_dir = data_input_dir, 
                              image_output_dir = image_output_dir, 
                              image_resolution = 100, 
                              save_images = True)
